### Pair Programming - ETL III

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
import requests
import pandas as pd
import numpy as np
import ast

from datetime import datetime, timedelta
pd.options.display.max_columns = None

In [2]:
class etl:
    def __init__(self,  longitud, latitud, pais, producto):

        self.longitud = longitud
        self.latitud = latitud
        self.pais = pais
        self.producto = producto



    def llamada(self):

        url= f'http://www.7timer.info/bin/api.pl?lon={self.longitud}&lat={self.latitud}&product={self.producto}&output=json'

        response = requests.get(url)
        codigo = response.status_code
        razon = response.reason

        print(f"El código de respuesta es {codigo}. Y la razon {razon}.")

        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
        df["country"] = self.pais
        
        return df



    def limpiar(self, df):

        df.reset_index()

        df["country"] = df["country"].str.lower()

        df_rh = df["rh_profile"].apply(pd.Series)

        for i in range(len(df_rh.columns)):
    
            nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 

            valores = list(df_rh[i].apply(pd.Series)["rh"])
    
            df.insert(i, nombre, valores)

        df_wind= df["wind_profile"].apply(pd.Series)

        for i in range(len(df_wind.columns)): 

            nombre = "wind_direction_" + str(df_wind[i].apply(pd.Series)["layer"][0])    
            
            valor1 = list(df_wind[i].apply(pd.Series)["direction"])     
        
            df.insert(i, nombre, valor1)                        
         

        for i in range(len(df_wind.columns)): 
  
            nombre2 = "wind_speed_" + str(df_wind[i].apply(pd.Series)["layer"][0])    
               
            valor2 = list(df_wind[i].apply(pd.Series)["speed"]) 

            df.insert(i, nombre2, valor2)

        df = df.drop(columns=["rh_profile", "wind_profile"], axis= 1)

        return df



    def medias(self,df):

        df_media = df.groupby("country").mean()
        return df_media



    def unir_datas(self, df1, df2):   

        df_total = df1.merge(df2, on="country") 
        return df_total

        

    def guardado(sef, df, ruta, nombre_fichero):   

        df.to_csv(f"{ruta}/{nombre_fichero}")

        return "Se ha guardado tu fichero."
    

In [3]:
meteorologico = etl(-100.445882, 39.7837304, "USA","meteo")
meteorologico

In [4]:
df_meteorologico = meteorologico.llamada()
df_meteorologico.head(2)

El código de respuesta es 200. Y la razon OK.


,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,USA
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1031,none,0,1,330,3,USA


In [5]:
df_meteorologico_limpio = meteorologico.limpiar(df_meteorologico)

In [6]:
df_meteorologico.head(2)

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,5,5,5,5,6,5,5,5,4,5,5,5,5,6,8,-9999,350,350,345,335,335,340,340,335,315,295,300,295,290,325,320,-9999,9,6,5,3,2,0,0,1,13,16,16,16,13,-2,-3,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,usa
1,-9999,3,5,5,5,5,5,5,5,5,5,5,5,7,8,8,-9999,340,345,340,335,335,340,335,325,320,320,320,325,350,335,325,-9999,10,7,6,4,1,-1,1,5,10,9,11,12,1,0,-3,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1031,none,0,1,330,3,usa


In [7]:
df_meteorologico_limpio.head()

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,5,5,5,5,6,5,5,5,4,5,5,5,5,6,8,-9999,350,350,345,335,335,340,340,335,315,295,300,295,290,325,320,-9999,9,6,5,3,2,0,0,1,13,16,16,16,13,-2,-3,3,5,-9999,-9999,-9999,-2,15,12,1032,none,0,-9999,300,3,usa
1,-9999,3,5,5,5,5,5,5,5,5,5,5,5,7,8,8,-9999,340,345,340,335,335,340,335,325,320,320,320,325,350,335,325,-9999,10,7,6,4,1,-1,1,5,10,9,11,12,1,0,-3,6,6,-9999,-9999,-9999,3,15,7,1031,none,0,1,330,3,usa
2,-9999,3,3,4,5,5,5,5,5,6,6,7,8,10,9,10,-9999,315,315,325,325,325,330,325,330,335,335,325,330,335,345,320,-9999,8,11,6,3,1,-1,1,2,-2,-3,-3,1,4,2,-2,9,2,-9999,-9999,-9999,5,10,6,1029,none,0,0,305,3,usa
3,-9999,3,3,4,5,5,5,5,6,7,9,10,12,13,12,13,-9999,315,310,320,320,320,315,325,335,335,340,335,335,340,350,320,-9999,9,8,4,1,-1,-2,1,-2,-2,-2,4,9,16,14,2,12,3,-9999,-9999,-9999,1,15,10,1030,none,0,0,295,2,usa
4,-9999,2,3,4,4,5,5,6,6,7,9,10,10,11,11,13,-9999,290,295,310,320,320,315,330,340,340,340,335,345,340,340,315,-9999,11,10,8,4,0,0,-2,-3,-1,2,6,0,3,6,5,15,4,-9999,-9999,-9999,-1,15,12,1031,none,0,0,275,2,usa


In [8]:
df_meteorologico_limpio_medias = meteorologico.medias(df_meteorologico_limpio)

/tmp/ipykernel_6939/2748524385.py:71: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_media = df.groupby("country").mean()


In [9]:
df_meteorologico_limpio_medias

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
usa,-9999.0,-152.734375,4.015625,4.0625,4.0625,4.109375,4.265625,4.625,4.984375,5.328125,5.828125,6.359375,7.046875,7.78125,8.78125,9.296875,-9999.0,53.296875,229.453125,241.25,248.125,256.5625,261.953125,274.765625,273.125,271.796875,268.4375,266.484375,259.53125,264.921875,264.21875,262.65625,-9999.0,-148.375,6.8125,5.3125,5.46875,5.515625,5.75,6.25,6.8125,7.078125,7.4375,9.140625,10.25,10.0625,6.265625,2.515625,97.5,6.703125,-9999.0,-9999.0,-9999.0,2.28125,13.34375,9.28125,1009.59375,0.796875,-468.09375,2.71875


In [10]:
meteorologico.guardado(df_meteorologico_limpio_medias, "datos", "prueba_guardado.csv")

'Se ha guardado tu fichero.'